##### Detecting Pneumonia disease with SVM. I have used Sigmoid function. There are a total of 40 chest x-ray images where 20 images are Pneumonia chest x-ray and the other 20 images are Healthy chest x-ray. 80% of the chest x-ray images have been used for training purpose and the other 20% of the images have been used for testing purpose.

#### Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd 
import random
import cv2
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import warnings

In [2]:
from random import shuffle 
from tqdm import tqdm 
from PIL import Image
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization, Conv2D, SeparableConv2D, MaxPool2D, LeakyReLU, Activation
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.svm import SVC

#### Dataset

In [3]:
#Importing dataset

in_path = './Dataset/'

In [4]:
def extract_data(dimensions, batch_length):
    tgen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, vertical_flip=True)
    test_val_datagen = ImageDataGenerator(rescale=1./255)
    tgen_final = tgen.flow_from_directory(directory=in_path+'train', target_size=(dimensions, dimensions), batch_size=batch_length, class_mode='binary', shuffle=True)
    test_gen = test_val_datagen.flow_from_directory(directory=in_path+'test', target_size=(dimensions, dimensions), batch_size=batch_length, class_mode='binary', shuffle=True)

    test_data = []
    test_labels = []
    
    train_data = []
    train_labels = []

    for normal_image in (os.listdir(in_path + 'test' + '/NORMAL/')):
        normal_image = plt.imread(in_path+'test'+'/NORMAL/'+normal_image)
        normal_image = cv2.resize(normal_image, (dimensions, dimensions))
        normal_image = normal_image.astype('float32') / 255
        label = 0
        test_data.append(normal_image)
        test_labels.append(label)

    for pneumonia_image in (os.listdir(in_path + 'test' + '/PNEUMONIA/')):
        pneumonia_image = plt.imread(in_path+'test'+'/PNEUMONIA/'+pneumonia_image)
        pneumonia_image = cv2.resize(pneumonia_image, (dimensions, dimensions))
        pneumonia_image = pneumonia_image.astype('float32') / 255
        label = 1
        test_data.append(pneumonia_image)
        test_labels.append(label)

    for normal_image in (os.listdir(in_path + 'train' + '/NORMAL/')):
        if normal_image == '.DS_Store':
            continue
        normal_image = plt.imread(in_path+'train'+'/NORMAL/'+normal_image)
        normal_image = cv2.resize(normal_image, (dimensions, dimensions))
        normal_image = normal_image.astype('float32') / 255
        label = 0
        train_data.append(normal_image)
        train_labels.append(label)

    for pneumonia_image in (os.listdir(in_path + 'train' + '/PNEUMONIA/')):
        if pneumonia_image == '.DS_Store':
            continue
        pneumonia_image = plt.imread(in_path+'train'+'/PNEUMONIA/'+pneumonia_image)
        pneumonia_image = cv2.resize(pneumonia_image, (dimensions, dimensions))
        pneumonia_image = pneumonia_image.astype('float32') / 255
        label = 1
        train_data.append(pneumonia_image)
        train_labels.append(label)
    
    return tgen_final, test_gen, test_data, test_labels, train_data, train_labels

In [5]:
img_dims = 100
epochs = 10
batch_size = 20

train_gen, test_gen, test_data, test_labels, train_data, train_labels = extract_data(img_dims, batch_size)

Found 32 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [6]:
flat_train_data = []
flat_test_data = []

for img in train_data: 
    flat_train_data.append(img.flatten())
    
for img in test_data: 
    flat_test_data.append(img.flatten())

In [7]:
listed_train_data = []
listed_test_data = []

for d in flat_train_data:
    listed_train_data.append(d.tolist()[:150*150])
    
for d in flat_test_data:
    listed_test_data.append(d.tolist()[:150*150])

In [8]:
train_labels_mini = train_labels
test_labels_mini = test_labels

#### Support Vector Machine (SVM)

In [9]:
clf = svm.SVC(kernel='sigmoid')
prediction = clf.fit(listed_train_data, train_labels_mini)

In [10]:
correctlyPredicted = 0
fp = 0
tp = 0
fn = 0
tn = 0

for ind in range(len(listed_test_data)): 
    pred = clf.predict([listed_test_data[ind]])
    real = test_labels_mini[ind]
    if real == 1 and pred == 1:
        tp += 1
    if real == 1 and pred == 0:
        fn += 1
    if real == 0 and pred == 1:
        fp += 1
    if real == 0 and pred == 0:
        tn += 1
    if pred == real:
        correctlyPredicted += 1

acc = correctlyPredicted/float(len(listed_test_data))*100
print("TEST DATA________________________")
print("Accuracy: {}".format(acc))
print("Confusion matrix: f_p = {}, t_p = {}, f_n = {}, t_n = {}".format(fp, tp, fn, tn))

TEST DATA________________________
Accuracy: 100.0
Confusion matrix: f_p = 0, t_p = 4, f_n = 0, t_n = 4


In [11]:
precision = tp/(tp+fp)
recall = tp/(tp+fn)
print("Precision: {}, Recall: {}".format(precision, recall))

Precision: 1.0, Recall: 1.0
